# Testing CBOR implementations for map encoding canonicality

Tested:
- [Julia](https://github.com/JuliaIO/CBOR.jl)
- [Rust](https://docs.rs/serde_cbor/0.11.0/serde_cbor/)
- [JS cbor](https://www.npmjs.com/package/cbor)
- [JS borc](https://www.npmjs.com/package/borc)

In [12]:
using CBOR, JSON

In [34]:
data_dict = Dict{String, UInt8}("a" => 1, "b" => 2)
data_json = json(data_dict)

"{\"b\":2,\"a\":1}"

In [51]:
encoded_jl = encode(data_dict);

In [52]:
encoded_js = JSON.parse(open(`node js/cbor.js borc encode $data_json`), inttype=UInt8)

7-element Array{Any,1}:
 0xa2
 0x61
 0x61
 0x01
 0x61
 0x62
 0x02

In [49]:
run(`node js/cbor.js borc decode $(json(encoded_jl))`)

{ b: 2, a: 1 }


Process(`node js/cbor.js borc decode '[162,97,98,24,2,97,97,24,1]'`, ProcessExited(0))

In [50]:
run(`node js/cbor.js borc decode $(json(encoded_js))`)

{ a: 1, b: 2 }


Process(`node js/cbor.js borc decode '[162,97,97,1,97,98,2]'`, ProcessExited(0))

In [56]:
encoded_rs = JSON.parse(open(`./rust/target/debug/rust encode $data_json`), inttype=UInt8)

7-element Array{Any,1}:
 0xa2
 0x61
 0x61
 0x01
 0x61
 0x62
 0x02